## Let’s harness some of the benefits of Infrastructure as Code

One of the superpowers of IaC is its repeatability, the fact that you can set something up once and then use the same information in multiple ways. Implementing IaC allows organizations to store configuration files describing the desired infrastructure as a single source of truth. It also allows you to apply the DevOps methodology that’s already in place for application code directly to the infrastructure. For example, configuration files can be stored and managed through GitHub using the same way your DevOps team manages the application code. This concept is often called “Shifting Left”, as you are describing the infrastructure to host an application earlier (left) in the delivery pipeline of the application. This allows for easier and consistent deployments of infrastructure across the complete infrastructure landscape of an organization.

## HPE GreenLake

HPE GreenLake is HPE’s edge-to-cloud platform. The HPE GreenLake platform provides a unified experience wherever your applications and its data are located on the edge, in colocations or in your own datacenter. This cloud experience everywhere includes the following capabilities:

* Self-service
* Infinite scalability
* Pay-as-you-go
* Managed for you

## HPE GreenLake Cloud Services

The HPE GreenLake ecosystem provides solutions for several top workloads such as containers, Machine Learning, private cloud, virtual machines, SAP HANA, HPC, VDI and many more. This page on [HPE GreenLake cloud services and ecosystem](https://www.hpe.com/us/en/greenlake/services.html) provides a complete list. The ecosystem also leverages many technologies from HPE partners such as Microsoft, VMware, SAP, Nutanix, Veeam and others. 

## HPE GreenLake for private cloud

One of the options provided by HPE GreenLake is to make it easy for customers to order and operate a private cloud with a mix of virtual machines, containers, and physical servers. This is exactly what the private cloud Service is all about. This service allows customers to point and click to create resources such as virtual machines. It also provides access via a public API, allowing developers to use  an Infrastructure-as-Code type of tool to automate provisioning, for example using Terraform.

## Terraform

[Terraform](https://terraform.io) is an open source Infrastructure-as-Code framework originally created by HashiCorp that is written in Go. It uses a declarative language (HashiCorp Configuration Language HCL or JSON more recently) to describe the desired state of the infrastructure in terms of cloud, virtual machines, networks, storage, and many other components. Terraform uses the concept of “providers” to integrate with all major public clouds. Terraform is a so-called idempotent system in the sense that it doesn’t generate any side effects if applied multiple times on an infrastructure already in its desired state. Terraform has gained quite the momentum in the last few years. Its main competition is Ansible, Amazon Cloud Formation, Puppet and Chef.


## Terraform installation

Your first step is to get your system ready to run Terraform. This will include:

1. Installing Terraform: follow [these steps](https://learn.hashicorp.com/tutorials/terraform/install-cli)
2. Verifying installation: **terraform --help**

We have already installed Terraform in you workshop environment, so you are ready to start building your infrastructure description file. Before we do so let's check the Terraform version installed on our system:

In [1]:
# Let's find out the version installed on this environment
terraform version

Terraform v1.1.9
on linux_amd64


## Building a Terraform configuration file from scratch

Let’s start building this TF file. It's called config.tf and you can open it for edition by double clicking from the left pane file explorer view. We will edit this file during the entire workshop, so keep it open and don't forget to save using the save icon from the taskbar above your notebooks. In case of issue with the file, you can find valid files at different stages from the solutions folder. Just cut/paste content from any of these to get back on track.

> Note: config.tf should be empty at this time.

### Selecting a Terraform provider

The first section of the file will enumerate the “providers” you rely upon for building your infrastructure, and they could be multiple providers in a single TF file. In this case here, you will only have the HPE GreenLake provider referenced as hpe/hpegl in the official [Terraform registry](https://registry.terraform.io/).

You can find out more about the HPE GreenLake Terraform provider from its [Terraform Registry page](https://registry.terraform.io/providers/HPE/hpegl/latest).

This page also provides a link to the GitHub repository corresponding to this provider. The [docs](https://github.com/hpe/terraform-provider-hpegl/tree/main/docs) folder is your best source of information for using the different data sources and resources provided by the provider. If you navigate to the resources section, you will see that one resource you can manipulate with this provider is a [VM instance](https://github.com/hpe/terraform-provider-hpegl/blob/main/docs/resources/vmaas_instance.md). Let’s focus on this resource in this article.

> Note: Because this is open source, don’t hesitate to open issues, or even a pull request, if you identify an issue.

Insert the following lines in your Terraform configuration file: config.tf and save it.

```json
# Load HPE GreenLake terraform provider
terraform {
      required_providers {
         hpegl = {
            source  = "hpe/hpegl"
            version = "0.1.7"
         }
      }
   }
```



### Terraform init

Before you can use Terraform, you will have to initialize it from the configuration file we have created. This is done with the following step: **terraform init**


In [1]:
# Let's inititalize our Terraform environment
terraform init


Initializing the backend...

Initializing provider plugins...
- Finding hpe/hpegl versions matching "0.1.7"...
- Installing hpe/hpegl v0.1.7...
- Installed hpe/hpegl v0.1.7 (self-signed, key ID D1F277A1AC66CE3D)

Partner and community providers are signed by their developers.
If you'd like to know more about provider signing, you can read about it here:
https://www.terraform.io/docs/cli/plugins/signing.html

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command t

#### Configuring the Terraform provider

Now that you have expressed the fact that the hpegl provider will be used, you need to setup some parameters for it. As explained on this [page](https://github.com/hpe/terraform-provider-hpegl/blob/main/docs/index.md), you can either explicitly set those parameters in your Terraform configuration file (tf file), or have them set in a series of environment variables, or a mix of both. 

I suggest the following two parameters be added in your TF file by cut/pasting it:

```json
# Setup provider environment (location and space)
provider "hpegl" {
      vmaas {
         location   = "HPE"
         space_name = "TerraForm Space"
      }
}
```

You can find your location and your space name from the HPE GreenLake user interface. In our example shown below, HPE is our location:

![GreenLake for private cloud](./Pictures/greenlakeforprivatecloud.png "GreenLake for private cloud")

And in the capture below, **Terraform Space** is the space we have created for our work with Terraform. You can check your available Spaces from the HPE GreenLake console under your profile icon, **Change space**.

![GreenLake select new space](./Pictures/greenlakeselectingspace.png "GreenLake select new space")

And the rest (such as tenant id, user id and user secret key) can be placed in a RC file, which you can source before running your Terraform command. 

We have already created this student.rc file for you, can take a look at it and source it to declare the variables:

In [2]:
# Check the content of student.rc
cat student.rc
# and source it
source ./student.rc
source ./didier.rc

export HPEGL_TENANT_ID={{ GREENLAKE_TENANT_ID }}
export HPEGL_USER_ID= {{ STUDENT_CLIENTID }}
export HPEGL_USER_SECRET= {{ STUDENT_CLIENTSECRET }}
export HPEGL_IAM_SERVICE_URL={{ GREENLAKE_IAM_API_URL  }}/oauth2/{{ GREENLAKE_TENANT_ID  }}/{{ GREENLAKE_IAM_ID }}
export TF_CLI_ARGS_plan="-parallelism=1"
export TF_CLI_ARGS_apply="-parallelism=1"
#export TF_LOG=DEBUG
export TF_LOG=bash: export: `}}': not a valid identifier
bash: export: `{{': not a valid identifier
bash: export: `}}': not a valid identifier
bash: export: `{{': not a valid identifier
bash: export: `}}': not a valid identifier
bash: export: `}}/oauth2/{{': not a valid identifier
bash: export: `}}/{{': not a valid identifier
bash: export: `}}': not a valid identifier


These values were all retrieved from the HPE GreenLake user interface. You can connect to your [console on HPE GreenLake](https://greenlake.hpe.com), using the same credentials as the ones used to connect to your Workshop-on-Demand. Feel free to give it a try.

But in a nutshell, your API Client access (which we created for you) can be retrieved from the HPE GreenLake console under your settings icon, **Identity & Access** and then the **API Clients** tab.

![GreenLake API Clients](./Pictures/greenlakeapiclients.png "GreenLake API Clients")

> Note: You need to remember the API Client secret key, as it’s not displayed anymore after creation.

Your client id and the Issuer URL are also available there as shown in capture below. 

![GreenLake Terraform API Client](./Pictures/greenlaketerraformapiclient.png "GreenLake Terraform API Client")


Finally, your Tenant ID is shown in the HPE GreenLake console under your profile icon, **API Access**.

![GreenLake Tenant ID](./Pictures/greenlaketenantid.png "GreenLake Tenant ID")



### Querying for infrastructure components

Your next step with the TF file is to query the HPE GreenLake provider to collect information needed to create your first VM instance. From the [documentation](https://github.com/hpe/terraform-provider-hpegl/blob/main/docs/resources/vmaas_instance.md), you can see that you need to gather the following information:

* Cloud ID
* Group ID
* Layout ID
* Plan ID
* Instance type code
* Network ID
* Resource Pool ID
* Template ID
* Folder Code

For this, you will use the Terraform **data** statements. For example, the following statement retrieves the Cloud ID and stores it (in variable called **cloud**), which we can later retrieve using: **data.hpegl_vmaas_cloud.cloud.id**:

```json
# Retrieve cloud id
data "hpegl_vmaas_cloud" "cloud" {
  name = "HPE GreenLake VMaaS Cloud-Trial4 "
   }
```

Cut it and paste it in your config.tf file. 

Then using a similar technique, you can retrieve the rest of the data you need. 

```json
# And a few networks
data "hpegl_vmaas_network" "blue_net" {
  name = "Blue-Network"
   }
data "hpegl_vmaas_network" "green_net" {
  name = "Green-network"
   }
 
data "hpegl_vmaas_cloud_folder" "compute_folder" {
   cloud_id = data.hpegl_vmaas_cloud.cloud.id
   name = "ComputeFolder"
   }
 
# Locate a resource pool
data "hpegl_vmaas_resource_pool" "cl_resource_pool" {
  cloud_id = data.hpegl_vmaas_cloud.cloud.id
  name = "ComputeResourcePool"
   }
 
# And a group
data "hpegl_vmaas_group" "default_group" {
  name = "HPEDEV-HackShackTenant-Group"
}
 
# Locate a plan
data "hpegl_vmaas_plan" "g1_small" {
  name = "G1-Small"
   }
 
# A layout
data "hpegl_vmaas_layout" "vmware" {
  name           	= "VMware VM with vanilla CentOS"
  instance_type_code = "glhc-vanilla-centos"
}
 
# And a template
data "hpegl_vmaas_template" "vanilla" {
  name = "vanilla-centos7-x86_64-09072020"
   }
```

Cut and paste this into you config file. The order of the statements doesn't really matter.

> You can get information about each of the data statements supported by the hpegl provider from [GitHub](https://github.com/hpe/terraform-provider-hpegl/tree/main/docs/data-sources).

### Validating the config file

Now that we have added a fair amount of content in our configuration file, a good practice is to validate regularly using the **terraform validate** command: 

In [7]:
# Validate config file 
terraform validate


Success! The configuration is valid.



### Creating a VM resource

The last step is to use a Terraform **resource** statement to request the creation of a new VM instance. Let's cut/paste the following resource statement to your configuration file:

```json
resource "hpegl_vmaas_instance" "DidierTest1" {
  name           	= "didou"
  cloud_id       	= data.hpegl_vmaas_cloud.cloud.id
 	group_id       	= data.hpegl_vmaas_group.default_group.id
 	layout_id      	= data.hpegl_vmaas_layout.vmware.id
 	plan_id        	= data.hpegl_vmaas_plan.g1_small.id
 	instance_type_code = data.hpegl_vmaas_layout.vmware.instance_type_code
 
 	network {
     	id = data.hpegl_vmaas_network.green_net.id
 	}
 
 	volume {
     	name     	= "root_vol"
     	size     	= 15
     	datastore_id = "auto"
 	}
 
 	config {
     	resource_pool_id = data.hpegl_vmaas_resource_pool.cl_resource_pool.id
     	template_id  	= data.hpegl_vmaas_template.vanilla.id
     	no_agent     	= true
     	asset_tag    	= "vm_terraform"
     	folder_code  	= data.hpegl_vmaas_cloud_folder.compute_folder.code
 	}
 
 	power = "poweron"
   }
 
```

> Note: You can get information about each of the resource statements supported by the hpegl provider from [GitHub](https://github.com/hpe/terraform-provider-hpegl/tree/main/docs/resources).

### Terraform ready to plan

I recommend running the **terraform validate** command again as you add sections to your file to track syntax errors. Once ready, the **terraform plan** command will provide information about what will be created when the **terraform apply** method is finally used.

Let's try this:
 

In [10]:
# Let's validate
terraform validate

# Let's see the plan
terraform plan

Success! The configuration is valid.


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # hpegl_vmaas_instance.DidierTest1 will be created
  + resource "hpegl_vmaas_instance" "DidierTest1" {
      + cloud_id           = 1
      + containers         = (known after apply)
      + group_id           = 3
      + history            = (known after apply)
      + hostname           = (known after apply)
      + id                 = (known after apply)
      + instance_type_code = "glhc-vanilla-centos"
      + layout_id          = 1159
      + name               = "didou"
      + plan_id            = 402
      + power              = "poweron"
      + server_id          = (known after apply)
      + status             = (known after apply)

      + config {
          + asset_tag        = "vm_terraform"
          + folder_code      = "group-v41"
 

If you agree with the plan, and what is going to be created, you can move to the last step, i.e. applying the configuration.

### Terraform ready to apply

The command you need to use is now: **terraform apply**. This will rerun the plan command, then apply it. It will prompt you to confirm before it starts building what’s in the plan. However for usage in a Jupyter Notebook environment we will 

In [3]:
terraform apply -input=false -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # hpegl_vmaas_instance.student1737 will be created
  + resource "hpegl_vmaas_instance" "student1737" {
      + cloud_id           = 1
      + containers         = (known after apply)
      + group_id           = 3
      + history            = (known after apply)
      + hostname           = (known after apply)
      + id                 = (known after apply)
      + instance_type_code = "glhc-vanilla-centos"
      + layout_id          = 1159
      + name               = "student1737"
      + plan_id            = 402
      + power              = "poweron"
      + server_id          = (known after apply)
      + status             = (known after apply)

      + config {
          + asset_tag        = "vm_terraform"
          + folder_code      = "group-v41"
          + no_agent         = tr

If we now check the list of VM in your GreenLake Console we should see our new VM, similar to what is shown below:

![GreenLake VM created using Terraform](./Pictures/vmcreated.png "GreenLake VM created using Terraform")



This concludes the first part of this workshop. We have been able to create a VM instance using Terraform. The configuration file that we built together can be stored in a GitHub repository and handled the same way you handle application code. Welcome to the world of Infrastructure-as-Code!

Let's now move to the [second part](2-WKSHP-Terraform101-ManagingResources.ipynb) in which we will continue to modify the configuration file, to change the desired state of the virtual machine.
